In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt
import transformers
from transformers import *
from tokenizers import BertWordPieceTokenizer
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
# import cudf, cuml, cupy
import gc
import numpy as np
from sklearn.preprocessing import normalize
from scipy.spatial.distance import hamming
from IPython.display import clear_output
import imagehash

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
from cuml.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [ ]:
from tqdm import tqdm

In [ ]:
# groups = train.groupby('label_group')['posting_id'].apply(lambda x: ' '.join(x)).reset_index()
# groups.columns = ['label_group', 'posting_id_list']
# train = train.merge(groups)

In [ ]:
import cudf
import re


def string_filter(line: str):
    for char in '?-*.!/\;:':  
        line = line.replace(char,'')
    return line

# train = pd.read_csv('../input/shopee-product-matching/train.csv')
# train['title'] = train['title'].apply(lambda x: x.lower())
# train['title'] = train['title'].apply(lambda x: string_filter(x))

# groups = train.groupby('label_group')['posting_id'].apply(lambda x: ' '.join(x)).reset_index()
# groups.columns = ['label_group', 'posting_id_list']
# train = train.merge(groups)
# train = cudf.from_pandas(train)

test = pd.read_csv('../input/shopee-product-matching/test.csv')
test['title'] = test['title'].apply(lambda x: x.lower())
test['title'] = test['title'].apply(lambda x: string_filter(x))

# groups = test.groupby('label_group')['posting_id'].apply(lambda x: ' '.join(x)).reset_index()
# groups.columns = ['label_group', 'posting_id_list']
# test = test.merge(groups)
test = cudf.from_pandas(test)

# test.head()

In [ ]:
# text_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 5), max_features=10000)
text_vectorizer = TfidfVectorizer(
    stop_words="english",
    analyzer="char",
    ngram_range=(2, 5),
    max_features=10000
)

In [ ]:
texts = test['title'].fillna('')
# labels = test['label_group']


In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(
#     texts, 
#     labels, 
#     random_state=0, 
#     test_size=0.1)

In [ ]:


tfidf_matrix = text_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)

In [ ]:
from cuml.common.sparsefuncs import csr_row_normalize_l2
import cupy


def efficient_csr_cosine_similarity(query, tfidf_matrix, matrix_normalized=False):
    query = csr_row_normalize_l2(query, inplace=False)
    if not matrix_normalized:
        tfidf_matrix = csr_row_normalize_l2(tfidf_matrix, inplace=False)
    
    return tfidf_matrix.dot(query.T)

In [ ]:
text_df = test


def document_search(series, top_n=3):
    query_vecs = text_vectorizer.transform(series)
    query_vec = cupy.sparse.csr_matrix(query_vecs.toarray()[0])
    similarities = efficient_csr_cosine_similarity(query_vec, tfidf_matrix, matrix_normalized=True)
    similarities = similarities.todense().reshape(-1)
    best_idx = similarities.argsort()[-top_n:][::-1]

    pp = cudf.DataFrame({
        'posting_id': text_df['posting_id'].iloc[best_idx],
        'text': text_df['title'].iloc[best_idx],
        'distance': 1 - similarities[best_idx]
    })
    return pp

In [ ]:
# test_text = train['title'][:2]
# print(test_text)

# similarities = document_search(
#     series=test_text
# )

# print(similarities.to_pandas())

In [ ]:
from tqdm.notebook import tnrange

j = 0
all_similarities = []
tf_matches = []

for i in tnrange(2, test.shape[0]+2):
    srez = text_df['title'][j:i]
#     print(srez)
    similarities = document_search(
        series=srez
    ).to_pandas()
    top_dists = similarities['distance']
    top_texts = similarities['text']
    top_posting_ids = similarities['posting_id']
    indexes = similarities.index
    
    a = ' '.join([top_posting_ids[i] for i in indexes if top_dists[i] < 0.3])
    tf_matches.append(a)
    
#     all_similarities.append({"p_id": top_posting_id, "dist": top_dist, "id": index})
    j += 1
#     print(top_texts, top_dists, indexes, '\n')

In [ ]:
# train = train.to_pandas()
# test = test.to_pandas()

posting_ids = test['posting_id'].to_arrow().to_pylist()

In [ ]:
# tfidf_matches = []

# for match_dict in tqdm(all_similarities):
#     distance = match_dict['dist']
#     index = match_dict['id']
#     if distance < 0.3:
#         match = ' '.join([str(index)])
#         tfidf_matches.append(match)

In [ ]:
# f1_score = []
# for tfidf_match, posting_id in zip(tf_matches, posting_ids):
#     match = set(tfidf_match.split())
#     posting_id_list = set(posting_id.split())
#     precision = 1 - len(match - posting_id_list) / len(match)
#     recall = 1 - len(posting_id_list - match) / len(posting_id_list)
#     f1_score.append(2 * precision * recall / (precision + recall))
# f1_score = np.mean(f1_score)

In [ ]:
# f1_score

In [ ]:
submission = pd.DataFrame({'posting_id': posting_ids, 'matches': tf_matches})

In [ ]:
submission.to_csv('submission.csv', index=False)